In [1]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Ground State and Excited State of H2 Molecule using VQE and VQD
===============================================================

Understanding the ground state and excited state energies of quantum
systems is paramount in various scientific fields. The **ground state
energy** represents the lowest energy configuration of a system, crucial
for predicting its stability, chemical reactivity, and electronic
properties. **Excited state energies**, on the other hand, reveal the
system\'s potential for transitions to higher energy levels, essential
in fields like spectroscopy, materials science, and quantum computing.
Both ground and excited state energies provide insights into fundamental
properties of matter, guiding research in diverse areas such as drug
discovery, semiconductor physics, and renewable energy technologies.

In this demo, we solve this problem by employ two quantum algorithms,
the Variational Quantum Eigensolver [#Vqe] to find the energy of
the ground state, and the Variational Quantum Deflation [#Vqd] to
find the excited state based on the above result. We recommend readers
to familiarize themselves with the [VQE tutorial from
Pennylane](https://pennylane.ai/qml/demos/tutorial_vqe/).


Defining the Hydrogen molecule
==============================

The [datasets]{.title-ref} package from Pennylane makes it a breeze to
find the Hamiltonian and the Hartree Fock state of some molecules, which
fortunately contain $H_2$.


In [2]:
import jax
import optax
import pennylane as qml
from pennylane import numpy as np

jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)

h2_dataset = qml.data.load("qchem", molname="H2", bondlength=0.742, basis="STO-3G")
h2 = h2_dataset[0]
H, qubits = h2.hamiltonian, len(h2.hamiltonian.wires)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  4
The Hamiltonian is    (-0.22250914236600539) [Z2]
+ (-0.22250914236600539) [Z3]
+ (-0.09963387941370971) [I0]
+ (0.17110545123720225) [Z1]
+ (0.17110545123720233) [Z0]
+ (0.12051027989546245) [Z0 Z2]
+ (0.12051027989546245) [Z1 Z3]
+ (0.16584090244119712) [Z0 Z3]
+ (0.16584090244119712) [Z1 Z2]
+ (0.16859349595532533) [Z0 Z1]
+ (0.1743207725924201) [Z2 Z3]
+ (-0.04533062254573469) [Y0 Y1 X2 X3]
+ (-0.04533062254573469) [X0 X1 Y2 Y3]
+ (0.04533062254573469) [Y0 X1 X2 Y3]
+ (0.04533062254573469) [X0 Y1 Y2 X3]


The [hf\_state]{.title-ref} will contain the orbital config with the
lowest energy. Let\'s see what it is.


In [3]:
h2.hf_state

tensor([1, 1, 0, 0], dtype=int64, requires_grad=True)

In the Hartree Fock representation, a qubit with state $1$ means that
there is an electron occupying the respective orbital. Chemistry teaches
us that the first few orbitals config are
$1s^1, 1s^2, 1s^22s^1, 1s^22s^2, ...$. We can see that in $H_2$, we
start from the config where the two electrons occupy the lowest two
energy levels.

Let's also see the gates used to evolve the hf state to the ground state

In [4]:
h2.vqe_gates

[DoubleExcitation(0.27324054462951564, wires=[0, 1, 2, 3])]

In [5]:
excitation_angle = 0.27324054462951564

Setting expectation for VQE and VQD
===================================

Before any training takes place, let's first look at some of the
empirical measured value. The energy of an atom at $n$ th excitement
level is denoted as $E_n$. Unlike computer scientists, in this case
physicists starts the value of $n$ from $1$. It is because
$E_n=\frac{E_I}{n^2}$, where $E_I$ is the ionization energy.

-   

    Ground state energy:

    :   -   $H$ atom: $E_1=-13.6eV$
        -   $H_2$ molecule: $4.52 eV$ (source: [Florida State
            University](https://web1.eng.famu.fsu.edu/~dommelen/quantum/style_a/hmol.html))

-   

    1st level excitation energy

    :   -   $H$ atom: $E_2=\frac{-13.6}{4}=-3.4eV$
        -   Therefore, to transition from $E_1$ to $E_2$ for $H$ atom:
            we need $E_1-E_2=10.2eV$

There are two units here: $eV$ (electron volt) and $Ha$ (Hatree energy).
They both measure energy, just like Joule or calorie but in the scale
for basic particles.


In [6]:
def hatree_energy_to_ev(hatree: float):
    return hatree * 27.2107

Just like training a neural network, the VQE needs two ingredients to
make it works. First we need to define an Ansatz (which plays the role
of the neural network), then a loss function.


Generating the ground state from a dataset \-\-\-\-\--

Starting from the HF state `[1 1 0 0]`, we will use the Given rotation
ansatz below to generate the state with the lowest energy.


In [7]:
dev = qml.device("default.qubit", wires=qubits)


@qml.qnode(dev)
def circuit_expected(theta):
    qml.BasisState(h2.hf_state, wires=range(qubits))
    qml.DoubleExcitation(theta, wires=[0, 1, 2, 3])
    return qml.expval(H)

print(qml.draw(circuit_expected)(0))

0: ─╭|Ψ⟩─╭G²(0.00)─┤ ╭<𝓗>
1: ─├|Ψ⟩─├G²(0.00)─┤ ├<𝓗>
2: ─├|Ψ⟩─├G²(0.00)─┤ ├<𝓗>
3: ─╰|Ψ⟩─╰G²(0.00)─┤ ╰<𝓗>


Let's find the ground energy state

In [13]:
gs_energy = circuit_expected(excitation_angle)
gs_energy

tensor(-1.13637658, requires_grad=True)

Define the lost function
========================

Remember that the lost function is the second ingredient. We use the
second equation in [this
paper](https://www.nature.com/articles/s41524-023-00965-1).

$$C_1(\theta) = \left\langle\Psi(\theta)|\hat H |\Psi (\theta) \right\rangle + \beta | \left\langle \Psi (\theta)| \Psi_0 \right\rangle|^2$$

At first sight, it might raise some eyebrows for someone from an ML
background, because we define the loss function based on the predicted
and the ground truth. However, note that we do not have any ground truth
value here. In this context, a loss function is just a function that we
want to minimize.

We can then define a lost function using the VQE and VQD methods. The
power of VQD is due to the third postulate of quantum mechanics and the
fact that the eigenbasis are orthogonal. Therefore, once we find the
parameters through VQE, our loss function only penalized eigenvector in
the second term. For this purpose, we implement the function with a
quantum technique called [swap
test](https://en.wikipedia.org/wiki/Swap_test). Let\'s see it in action.


In [9]:
dev_swap = qml.device("default.qubit", wires=qubits * 2 + 1)


@qml.qnode(dev_swap)
def circuit_loss_2(param):
    """
    Constructs a quantum circuit for finding the excited state using swap test.

    Args:
    param (float): Rotation angle for the Double Excitation gate, to be optimized.
    theta_0 (float): The rotation angle corresponding to ground energy.

    Returns:
    Probability distribution of measurement outcomes on the 8th wire.

    """
    qml.BasisState(h2.hf_state, wires=range(0, qubits))
    qml.BasisState(h2.hf_state, wires=range(qubits, qubits * 2))
    for op in h2.vqe_gates:
        qml.apply(op)    
    qml.DoubleExcitation(param, wires=range(qubits, qubits * 2))
    qml.Hadamard(8)
    for i in range(0, qubits):
        qml.CSWAP([8, i, i + qubits])
    qml.Hadamard(8)
    return qml.probs(8)

Let's preview the circuit\...


In [10]:
print(qml.draw(circuit_loss_2)(param=1))

0: ─╭|Ψ⟩─╭G²(0.27)─╭SWAP──────────────────────┤       
1: ─├|Ψ⟩─├G²(0.27)─│─────╭SWAP────────────────┤       
2: ─├|Ψ⟩─├G²(0.27)─│─────│─────╭SWAP──────────┤       
3: ─╰|Ψ⟩─╰G²(0.27)─│─────│─────│─────╭SWAP────┤       
4: ─╭|Ψ⟩─╭G²(1.00)─├SWAP─│─────│─────│────────┤       
5: ─├|Ψ⟩─├G²(1.00)─│─────├SWAP─│─────│────────┤       
6: ─├|Ψ⟩─├G²(1.00)─│─────│─────├SWAP─│────────┤       
7: ─╰|Ψ⟩─╰G²(1.00)─│─────│─────│─────├SWAP────┤       
8: ──H─────────────╰●────╰●────╰●────╰●─────H─┤  Probs


The circuit consists of operations to prepare the initial states for the
excited and ground states of $H_2$, apply the Double Excitation gate
with the provided parameters, and the swap test. Here we reserve wires 0
to 3 for the excited state calculation and wires 4 to 7 for the ground
state of $H_2$.

Now we will define the loss functions. The first
([loss\_fn\_1]{.title-ref}) is using VQE to obtain the ground state
energy and the second ([loss\_fn\_2]{.title-ref}) use VQD to compute the
excited energy using the results obtained by optimizing for
[loss\_fn\_1]{.title-ref}.


In [11]:
def loss_f(theta, beta):
    measurement = circuit_loss_2(theta)
    return beta * (measurement[0] - 0.5) / 0.5


def optimize(beta):
    theta = 0.0

    # store the values of the cost function
    energy = [loss_f(theta, beta)]
    conv_tol = 1e-6
    max_iterations = 100
    opt = optax.sgd(learning_rate=0.4)

    # store the values of the circuit parameter
    angle = [theta]

    opt_state = opt.init(theta)

    for n in range(max_iterations):
        gradient = jax.grad(loss_f)(theta, beta)
        updates, opt_state = opt.update(gradient, opt_state)
        theta = optax.apply_updates(theta, updates)
        angle.append(theta)
        energy.append(circuit_expected(theta))

        conv = np.abs(energy[-1] - energy[-2])

        if n % 1 == 0:
            print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha, {theta}")

        if conv <= conv_tol:
            break
    return angle[-1], energy[-1]

We now have all we need to run the ground state and 1st excited state
optimization.


For the excited state, we are going to choose the value for $\beta$,
such that $\beta > E_1 - E_0$. In other word, $\beta$ needs to be larger
than the gap between the ground state energy and the first excited state
energy.


In [14]:
beta = 6

first_excite_theta, first_excite_energy = optimize(beta=beta)

hatree_energy_to_ev(gs_energy), hatree_energy_to_ev(first_excite_energy)

Step = 0,  Energy = -1.01803971 Ha, -0.3238238080027143
Step = 1,  Energy = -0.60333826 Ha, -0.9984843874025111
Step = 2,  Energy = 0.25053938 Ha, -2.145216985963448
Step = 3,  Energy = 0.47818493 Ha, -2.939303103162038
Step = 4,  Energy = 0.47691795 Ha, -2.854233326221313
Step = 5,  Energy = 0.47763624 Ha, -2.8711753026259768
Step = 6,  Energy = 0.47751113 Ha, -2.867787474727545
Step = 7,  Energy = 0.47753689 Ha, -2.868465035770822
Step = 8,  Energy = 0.47753177 Ha, -2.8683295235984567
Step = 9,  Energy = 0.47753280 Ha, -2.8683566260326394
Step = 10,  Energy = 0.47753259 Ha, -2.8683512055458054


(tensor(-30.9216021, requires_grad=True), Array(12.99399607, dtype=float64))

The result should produce something close to the first ionization energy
of $H_2$ is $1312.0 kJ/mol$ according to
[Wikipedia](https://en.wikipedia.org/wiki/Hydrogen). Note that this is
the ionization energy, at which the electron is completely removed from
the molecule. Here we are calculating the excited state energy, where an
electron moves to the outer shell only. Intuitively, we should a lower
number than above. We now see how close the result is to reality.


In [15]:
kj_per_mol_per_hatree = 2625.5
ground_truth_in_kj_per_mol = 1312
prediction_in_kj_per_mol = first_excite_energy * kj_per_mol_per_hatree

error = np.abs(prediction_in_kj_per_mol - ground_truth_in_kj_per_mol)
print(f"Predicted E_2 is {prediction_in_kj_per_mol}.")
print(
    f"The result is {error} kJ/mol different from reality, or {100 - (prediction_in_kj_per_mol / ground_truth_in_kj_per_mol * 100)} percent"
)

Predicted E_2 is 1253.7618176981923.
The result is 58.23818230180768 kJ/mol different from reality, or 4.438885846174372 percent


Conclusion
==========

We have used VQE and VQD to find the ground state and the excited state
of the $H_2$ molecule. One of the applications is in photovoltaic
devices. For example, the design of solar cells relies on optimizing the
energy levels of donor and acceptor materials to facilitate charge
separation and collection, thereby enhancing solar energy conversion
efficiency.

To build up on this work, we recommend readers to run this script with
more complex molecules and/or find the energy needed for higher
excitation levels. Also do not forget check out other tutorials for
Quantum chemistry here in Pennylane. Good luck on your Quantum chemistry
journey!


References
==========

About the author
================
